In [19]:
%matplotlib qt

# Energy calculator

Caculates the energy of a single Ho ion in the LiHoF<sub>4</sub> crystal.

In [1]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
from scipy import optimize
import math
import pandas as pd
import sys
import os

In [2]:
#constants
hbar=1
J=8
deg_J = 2 * J + 1
u_B=0.6717
g_L=5/4

In [39]:
# J matrices
jplus = hbar * np.diag(np.array( [ math.sqrt(J*(J+1) - m*(m+1)) for m in np.arange(-J,J) ] ), 1)
jminus = hbar * np.diag(np.array( [ math.sqrt(J*(J+1) - m*(m-1)) for m in np.arange(-J+1,J+1) ] ),-1)
jx = (jplus + jminus) * 0.5
jy = (jplus - jminus) * (-0.5j)
jz = hbar * np.diag(np.arange(-J,J+1))
I_J = np.diag(np.ones(int(round(deg_J))))


# crystal field equivalent operators
O02 = 3 * LA.matrix_power(jz,2) - J*(J+1)*I_J
O04 = 35 * LA.matrix_power(jz,4) - 30 * J * (J+1) * LA.matrix_power(jz,2) + 25*LA.matrix_power(jz,2) - 6 * J * (J+1) * I_J + 3 * J**2 * (J+1)**2 * I_J
O44C = 0.5 * (LA.matrix_power(jplus,4) + LA.matrix_power(jminus,4))
O06 = 231 * LA.matrix_power(jz,6) - 315*J*(J+1)*LA.matrix_power(jz,4) + 735*LA.matrix_power(jz,4) + 105 * J**2 * (J+1)**2 * LA.matrix_power(jz,2) - 525*J*(J+1)*LA.matrix_power(jz,2) + 294*LA.matrix_power(jz,2) - 5 * J**3 * (J+1)**3 * I_J + 40 * J**2 *(J+1)**2 * I_J - 60*J*(J+1)*I_J
O46C1 = 0.25 * (LA.matrix_power(jplus,4) + LA.matrix_power(jminus,4)) @ (11*LA.matrix_power(jz,2) - J*(J+1)*I_J - 38*I_J)
O46S1 = -0.25j * (LA.matrix_power(jplus,4) - LA.matrix_power(jminus,4)) @ (11 * LA.matrix_power(jz,2) - J*(J+1)*I_J - 38*I_J)
O46C = O46C1 + np.transpose(np.conj(O46C1))
O46S = O46S1 + np.transpose(np.conj(O46S1))


# crystal field parameters
B02 = -0.696
B04 = 4.06e-3
B06 =  4.64e-6
B44C = 0.0418
B46C = 8.12e-4
B46S = 1.137e-4


# crystal field Hamiltonian
H_cf = B02*O02 + B04*O04 + B06*O06 + B44C*O44C + B46C*O46C + B46S*O46S

In [104]:
# linear spacing for B:
Bx = np.linspace(-5,5,num=1601)
By = 0.0
Bz = 0.0

In [101]:
H = [H_cf - u_B*g_L*(bx*jx + By*jy + Bz*jz) for bx in Bx]    # zeeman term

In [102]:
res_up=[]
res_down=[]
for h in H:
    w,v = LA.eigh(h)
    res_up.append(w[0])
    res_down.append(w[1])
res_up = np.array(res_up)
res_down = np.array(res_down)

In [103]:
# plot energy decrease for a single ion when the transverse field is the average of a field in the 
# same direction of the external field and a field in the opposite direction. This is in comparison to no
# such fields - just the external field.
# This should provide a measure for the effectiveness of the ODD muchanism for any given external Bx.
delta_bx=0.03
delta_bx_idx = int(delta_bx/(Bx[1]-Bx[0]))
plt.plot(Bx[delta_bx_idx:-delta_bx_idx],(res_up - (np.roll(res_up,delta_bx_idx) + np.roll(res_up,-delta_bx_idx))*0.5)[delta_bx_idx:-delta_bx_idx])
plt.plot(Bx[delta_bx_idx:-delta_bx_idx],(res_down - (np.roll(res_down,delta_bx_idx) + np.roll(res_down,-delta_bx_idx))*0.5)[delta_bx_idx:-delta_bx_idx])
plt.plot(Bx[delta_bx_idx:-delta_bx_idx],(((res_up - (np.roll(res_up,delta_bx_idx) + np.roll(res_up,-delta_bx_idx))*0.5) + (res_down - (np.roll(res_down,delta_bx_idx) + np.roll(res_down,-delta_bx_idx))*0.5))*0.5)[delta_bx_idx:-delta_bx_idx])

In [95]:
plt.plot(Bx,res_up)
plt.plot(Bx,res_down)

In [85]:
plt.close()